
Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

global df
df = pd.DataFrame(doc, columns=['text'])


In [2]:

def correctif(val):
    if isinstance(val,str):
        if 'Jan' in val:
            return re.sub(r'(Jan\w+)', 'Jan',val)
        elif 'Feb' in val:
            return re.sub(r'(Feb\w+)', 'Feb',val)
        elif 'Mar' in val:
            return re.sub(r'(Mar\w+)', 'Mar',val)
        elif 'Apr' in val:
            return re.sub(r'(Apr\w+)', 'Apr',val)
        elif 'May' in val:
            return re.sub(r'(May\w*)', 'May',val)
        elif 'June' in val:
            return re.sub(r'(Jun\w*)', 'Jun',val)
        elif 'Jul' in val:
            return re.sub(r'(Jul\w+)', 'Jul',val)
        elif 'Aug' in val:
            return re.sub(r'(Aug\w+)', 'Aug',val)
        elif 'Sep' in val:
            return re.sub(r'(Sep\w+)', 'Sep',val)
        elif 'Oct' in val:
            return re.sub(r'(Oct\w+)', 'Oct',val)
        elif 'Nov' in val:
            return re.sub(r'(Nov\w+)', 'Nov',val)
        elif 'Dec' in val:
            return re.sub(r'(Dec\w+)', 'Dec',val)
        else:
            return val
    else:
        return val

In [3]:
pattern1 = re.compile(r'(\d{1,2}/\d{1,2}/\d{4}\b)')
pattern2 = re.compile(r'(\d{1,2}[/|-]\d{1,2}[/|-]\d{2}\b)')
pattern3 = re.compile(r'(\d/\d/\d{2}\b)')
pattern4 = re.compile(r'((?:\d+\W+)?(?:(?:Jan|Feb|March|Apr|May|Jun|July|Aug|Sep|Oct|Nov|Dec)\w*)[\s|-|,|\.]*\w+[\s|-|,|\s]*\d+)')
pattern5 = re.compile(r'(\d{1,2}/\d{4})\b')
pattern6 = re.compile(r'(\d{4})\b')
pattern_list = [pattern1,pattern2,pattern3,
                pattern4,pattern5,pattern6
               ]
dates = []
for text in df['text'].values:
    ind = 0
    while(len(pattern_list[ind].findall(text))==0):
        ind+=1
    dates.append(pattern_list[ind].findall(text)[0])
        
df['dates'] = dates
df['dates'] = df['dates'].apply(correctif)
df['dates'] = pd.to_datetime(df['dates'], infer_datetime_format=True)


In [6]:
def date_sorter():
    # Your code here
    data = df.sort_values(by='dates')
    return pd.Series(data.index)

In [7]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64